<a href="https://colab.research.google.com/github/YianKim/2022_uncertainty_aware_semisupervise/blob/main/uncertainty_%EA%B8%B0%EB%B0%98_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글 드라이브


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 라이브러리 불러오기


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import PIL
import pickle
import random
from tqdm import tqdm
from collections import Counter

from keras.layers.core import Lambda
from keras import backend as K

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import LSTM
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import *
from keras.models import load_model

from tqdm import tqdm

## 데이터 불러오기


In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(data, labels), (test_data, test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [4]:
train_data = data[range(10000)].reshape([10000,28,28,1])
train_labels = labels[range(10000)].reshape([10000,1])

In [5]:
unlab_data = data[range(10000,60000)].reshape([50000,28,28,1])
unlab_labels = labels[range(10000,60000)].reshape([50000,1])

In [6]:
test_data = test_data.reshape([10000,28,28,1])
test_labels = test_labels.reshape([10000,1])

In [7]:
train_labels2 = []
unlab_labels2 = []
test_labels2 = []

In [8]:
for i in range(10000):
  white = [0,0,0,0,0,0,0,0,0,0]
  white[train_labels[i][0]] = 1
  train_labels2.append(white)

for i in range(50000):
  white = [0,0,0,0,0,0,0,0,0,0]
  white[unlab_labels[i][0]] = 1
  unlab_labels2.append(white)

for i in range(10000):
  white = [0,0,0,0,0,0,0,0,0,0]
  white[test_labels[i][0]] = 1
  test_labels2.append(white)

In [9]:
train_labels = np.array(train_labels2)
unlab_labels = np.array(unlab_labels2)
test_labels = np.array(test_labels2)

## CNN

In [ ]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

In [ ]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))
model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

model.fit(
          train_data,
          train_labels,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/2000
110/110 [==============================] - 1s 9ms/step - loss: 2.7861 - accuracy: 0.6353 - val_loss: 0.8022 - val_accuracy: 0.7300 - lr: 0.0010
Epoch 2/2000
110/110 [==============================] - 1s 7ms/step - loss: 0.6622 - accuracy: 0.7697 - val_loss: 0.6885 - val_accuracy: 0.7657 - lr: 0.0010
Epoch 3/2000
110/110 [==============================] - 1s 6ms/step - loss: 0.5660 - accuracy: 0.7983 - val_loss: 0.6609 - val_accuracy: 0.7703 - lr: 0.0010
Epoch 4/2000
110/110 [==============================] - 1s 6ms/step - loss: 0.5037 - accuracy: 0.8180 - val_loss: 0.6342 - val_accuracy: 0.7733 - lr: 0.0010
Epoch 5/2000
110/110 [==============================] - 1s 9ms/step - loss: 0.4535 - accuracy: 0.8319 - val_loss: 0.6077 - val_accuracy: 0.7847 - lr: 0.0010
Epoch 6/2000
110/110 [==============================] - 1s 7ms/step - loss: 0.4147 - accuracy: 0.8441 - val_loss: 0.6093 - val_accuracy: 0.7960 - lr: 0.0010
Epoch 7/2000
110/110 [==============================] - 1s

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

## resnet

In [ ]:
from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

def residual_block(x: Tensor, downsample: bool, filters: int, kernel_size: int = 3) -> Tensor:
    y = Conv2D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

def create_res_net():
    
    inputs = Input(shape=(28, 28, 1))
    num_filters = 64
    
    t = BatchNormalization()(inputs)
    t = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)
    t = PermaDropout(0.5)(t)
    
    num_blocks_list = [2, 5, 5, 2]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            t = residual_block(t, downsample=(j==0 and i!=0), filters=num_filters)
        num_filters *= 2
        t = PermaDropout(0.5)(t)
    
    t = AveragePooling2D(4)(t)
    t = Flatten()(t)
    outputs = Dense(10, activation='softmax')(t)
    
    model = Model(inputs, outputs)

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
model = create_res_net() # or create_plain_net()
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 batch_normalization_186 (Batch  (None, 28, 28, 1)   4           ['input_8[0][0]']                
 Normalization)                                                                                   
                                                                                                  
 conv2d_197 (Conv2D)            (None, 28, 28, 64)   640         ['batch_normalization_186[0][0]']
                                                                                                  
 re_lu_179 (ReLU)               (None, 28, 28, 64)   0           ['conv2d_197[0][0]']       

In [ ]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')

model.fit(
    x=train_data,
    y=train_labels,
    epochs=500,
    verbose=1,
    validation_split=0.3,
    batch_size=64,
    callbacks=[lr_reducer, early_stopper]
)

Epoch 1/500
110/110 [==============================] - 33s 236ms/step - loss: 1.2659 - accuracy: 0.5479 - val_loss: 29.7893 - val_accuracy: 0.1130 - lr: 0.0010
Epoch 2/500
110/110 [==============================] - 25s 226ms/step - loss: 0.6934 - accuracy: 0.7389 - val_loss: 5.3014 - val_accuracy: 0.1453 - lr: 0.0010
Epoch 3/500
110/110 [==============================] - 25s 226ms/step - loss: 0.5644 - accuracy: 0.7911 - val_loss: 2.9814 - val_accuracy: 0.2857 - lr: 0.0010
Epoch 4/500
110/110 [==============================] - 25s 227ms/step - loss: 0.4975 - accuracy: 0.8141 - val_loss: 1.2112 - val_accuracy: 0.6627 - lr: 0.0010
Epoch 5/500
110/110 [==============================] - 25s 226ms/step - loss: 0.4525 - accuracy: 0.8384 - val_loss: 0.7615 - val_accuracy: 0.7643 - lr: 0.0010
Epoch 6/500
110/110 [==============================] - 25s 227ms/step - loss: 0.4306 - accuracy: 0.8421 - val_loss: 0.7135 - val_accuracy: 0.7747 - lr: 0.0010
Epoch 7/500
110/110 [========================

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_resnet.h5')

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


## 모델 결과

In [10]:
def model_eval():
  pred_mu = model.predict(test_data)
  for i in range(1, 30):
    pred_mu += model.predict(test_data)
  pred_mu = pred_mu/30
  predicted_test_labels = np.argmax(pred_mu, axis=1)
  return(accuracy_score(np.argmax(test_labels, axis=1), predicted_test_labels))

In [11]:
# model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_resnet.h5')
model_eval()

0.892

# 라벨링

## 라벨링; 1

In [12]:
# model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_resnet.h5')

In [13]:
# Vars : 분산들
# Outs : 결과 값들

Vars = []
Outs = []

temp1 = []

for i in tqdm(range(30)):
  temp1.append(model.predict(unlab_data))

for j in tqdm(range(unlab_data.shape[0])):

  temp2 = np.array([0,0,0,0,0,0,0,0,0,0], 'float32')
  
  for i in range(30):
    temp2 += temp1[i][j]
  Outs.append(temp2/30)
  
  outputs=[]

  for i in range(30):
    outputs.append(temp1[i][j][np.argmax(temp2)])
  Vars.append(np.var(outputs))

100%|██████████| 50000/50000 [00:09<00:00, 5417.50it/s]


In [14]:
Outs = np.array(Outs)

class마다 균등하게 선택

In [15]:
Counter(np.argmax(unlab_labels, axis=1))

Counter({0: 5058,
         1: 4973,
         2: 4984,
         3: 4981,
         4: 5026,
         5: 5011,
         6: 4979,
         7: 4978,
         8: 5010,
         9: 5000})

### 불확정성 컷 1

In [ ]:
# upper bound of variance?

UB = []

for h in range(10):
  classvars = []
  for i in tqdm(range(50000)):
    if np.argmax(unlab_labels, axis=1).tolist()[i]==h:
      classvars.append(Vars[i])
  UB.append(np.percentile(classvars, 50))

100%|██████████| 50000/50000 [01:24<00:00, 589.32it/s]


In [ ]:
lowvars = []
ind = 0 

for i in Vars:
  if i <= UB[np.argmax(unlab_labels, axis=1)[ind]]:
    lowvars.append(ind)
  ind += 1

In [ ]:
highvars = []
for i in range(unlab_data.shape[0]):
  if i not in lowvars:
    highvars.append(i)

In [ ]:
# 같은 길이의 랜덤하게 고른 data들에 모델로 label 부여 후 정확도 측정
randomindx = random.sample(range(50000), 50000)
randomindx2 = randomindx[0:25000]
randomindx = randomindx[25000:50000]
accuracy_score(np.argmax(np.array(Outs)[randomindx], axis=1), np.argmax(unlab_labels[randomindx], axis=1))

0.89192

In [ ]:
# 저분산의 data들만 모아서 모델로 label 부여 후 정확도 측정 : 95% 이상
accuracy_score(np.argmax(np.array(Outs)[lowvars], axis=1), np.argmax(unlab_labels[lowvars], axis=1))

0.9728878433874247

In [ ]:
train_data_1 = np.concatenate([train_data, unlab_data[lowvars]], axis=0)
train_labels_1 = np.concatenate([train_labels, Outs[lowvars]], axis=0)

In [ ]:
shufindx = random.sample(range(train_data_1.shape[0]),train_data_1.shape[0])
train_data_1 = train_data_1[shufindx]
train_labels_1 = train_labels_1[shufindx]

In [ ]:
train_data_2 = np.concatenate([train_data, unlab_data[randomindx]], axis=0)
train_labels_2 = np.concatenate([train_labels, Outs[randomindx]], axis=0)

In [ ]:
shufindx = random.sample(range(train_data_2.shape[0]),train_data_2.shape[0])
train_data_2 = train_data_2[shufindx]
train_labels_2 = train_labels_2[shufindx]

In [ ]:
unlab_data_1 = unlab_data[highvars]
unlab_labels_1 = unlab_labels[highvars]
unlab_data_2 = unlab_data[randomindx2]
unlab_labels_2 = unlab_labels[randomindx2]

In [ ]:
Counter(np.argmax(train_labels_1, axis=1))

Counter({0: 3600,
         1: 3599,
         2: 3613,
         3: 3638,
         4: 3469,
         5: 3491,
         6: 3151,
         7: 3507,
         8: 3505,
         9: 3508})

In [ ]:
Counter(np.argmax(train_labels_2, axis=1))

Counter({0: 3409,
         1: 3473,
         2: 3582,
         3: 3546,
         4: 3512,
         5: 3488,
         6: 3448,
         7: 3581,
         8: 3447,
         9: 3514})

In [ ]:
Counter(np.argmax(unlab_labels_1, axis=1))

Counter({0: 2529,
         1: 2407,
         2: 2492,
         3: 2490,
         4: 2513,
         5: 2505,
         6: 2489,
         7: 2489,
         8: 2505,
         9: 2500})

In [ ]:
Counter(np.argmax(unlab_labels_2, axis=1))

Counter({0: 2565,
         1: 2496,
         2: 2452,
         3: 2497,
         4: 2506,
         5: 2458,
         6: 2504,
         7: 2474,
         8: 2554,
         9: 2494})

### 불확정성 컷 2

In [16]:
# upper bound of variance?

UB25 = []
UB50 = []
UB75 = []


for h in range(10):
  classvars = []
  for i in tqdm(range(50000)):
    if np.argmax(unlab_labels, axis=1).tolist()[i]==h:
      classvars.append(Vars[i])
  UB25.append(np.percentile(classvars, 25))
  UB50.append(np.percentile(classvars, 50))
  UB75.append(np.percentile(classvars, 75))

100%|██████████| 50000/50000 [01:27<00:00, 569.78it/s]


In [17]:
# UB25 < UB50 < UB75

In [18]:
vars25 = []
vars50 = []
vars75 = []
vars100 = []

ind = 0 


for i in Vars:
  if i <= UB25[np.argmax(unlab_labels, axis=1)[ind]]:
    vars25.append(ind)
  elif i <= UB50[np.argmax(unlab_labels, axis=1)[ind]]:
    vars50.append(ind)
  elif i <= UB75[np.argmax(unlab_labels, axis=1)[ind]]:
    vars75.append(ind)
  else:
    vars100.append(ind)
  ind += 1

In [19]:
k1 = random.sample(range(len(vars25)), len(vars25))
k2 = random.sample(range(len(vars50)), len(vars50))
k3 = random.sample(range(len(vars75)), len(vars75))
k4 = random.sample(range(len(vars100)), len(vars100))

lowvars = k1[0:np.int(len(k1)/2)] + k2[0:np.int(len(k2)/2)] + k3[0:np.int(len(k3)/2)] + k4[0:np.int(len(k4)/2)]
highvars = k1[np.int(len(k1)/2):len(k1)] + k2[np.int(len(k2)/2):len(k2)] + k3[np.int(len(k3)/2):len(k3)] + k4[np.int(len(k4)/2):len(k4)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Depreca

In [20]:
# 같은 길이의 랜덤하게 고른 data들에 모델로 label 부여 후 정확도 측정
randomindx = random.sample(range(50000), 50000)
randomindx2 = randomindx[0:25000]
randomindx = randomindx[25000:50000]
accuracy_score(np.argmax(np.array(Outs)[randomindx], axis=1), np.argmax(unlab_labels[randomindx], axis=1))

0.89432

In [21]:
# 층화추출
accuracy_score(np.argmax(np.array(Outs)[lowvars], axis=1), np.argmax(unlab_labels[lowvars], axis=1))

0.8876755070202809

In [22]:
train_data_1 = np.concatenate([train_data, unlab_data[lowvars]], axis=0)
train_labels_1 = np.concatenate([train_labels, Outs[lowvars]], axis=0)

In [23]:
shufindx = random.sample(range(train_data_1.shape[0]),train_data_1.shape[0])
train_data_1 = train_data_1[shufindx]
train_labels_1 = train_labels_1[shufindx]

In [24]:
train_data_2 = np.concatenate([train_data, unlab_data[randomindx]], axis=0)
train_labels_2 = np.concatenate([train_labels, Outs[randomindx]], axis=0)

In [25]:
shufindx = random.sample(range(train_data_2.shape[0]),train_data_2.shape[0])
train_data_2 = train_data_2[shufindx]
train_labels_2 = train_labels_2[shufindx]

In [26]:
unlab_data_1 = unlab_data[highvars]
unlab_labels_1 = unlab_labels[highvars]
unlab_data_2 = unlab_data[randomindx2]
unlab_labels_2 = unlab_labels[randomindx2]

In [27]:
Counter(np.argmax(train_labels_1, axis=1))

Counter({0: 3539,
         1: 3508,
         2: 3750,
         3: 3530,
         4: 3372,
         5: 3486,
         6: 3373,
         7: 3539,
         8: 3401,
         9: 3501})

In [28]:
Counter(np.argmax(train_labels_2, axis=1))

Counter({0: 3497,
         1: 3448,
         2: 3693,
         3: 3566,
         4: 3372,
         5: 3444,
         6: 3363,
         7: 3591,
         8: 3444,
         9: 3582})

In [29]:
Counter(np.argmax(unlab_labels_1, axis=1))

Counter({0: 2507,
         1: 2456,
         2: 2436,
         3: 2505,
         4: 2490,
         5: 2529,
         6: 2595,
         7: 2522,
         8: 2389,
         9: 2572})

In [30]:
Counter(np.argmax(unlab_labels_2, axis=1))

Counter({0: 2559,
         1: 2517,
         2: 2497,
         3: 2470,
         4: 2534,
         5: 2471,
         6: 2491,
         7: 2484,
         8: 2549,
         9: 2428})

## 모델링

간단한 모델에서 받은 지식를 복잡한 모델에서 학습 

uncertainty aware data vs random sampling data

#### CNN

In [ ]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

In [ ]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)


model.fit(
          train_data_1,
          train_labels_1,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/2000
383/383 [==============================] - 5s 12ms/step - loss: 1.1925 - accuracy: 0.6126 - val_loss: 0.6793 - val_accuracy: 0.7716 - lr: 0.0010
Epoch 2/2000
383/383 [==============================] - 4s 10ms/step - loss: 0.6240 - accuracy: 0.7889 - val_loss: 0.5945 - val_accuracy: 0.7977 - lr: 0.0010
Epoch 3/2000
383/383 [==============================] - 4s 10ms/step - loss: 0.5377 - accuracy: 0.8277 - val_loss: 0.5035 - val_accuracy: 0.8390 - lr: 0.0010
Epoch 4/2000
383/383 [==============================] - 4s 10ms/step - loss: 0.4882 - accuracy: 0.8422 - val_loss: 0.4723 - val_accuracy: 0.8481 - lr: 0.0010
Epoch 5/2000
383/383 [==============================] - 4s 10ms/step - loss: 0.4643 - accuracy: 0.8558 - val_loss: 0.4740 - val_accuracy: 0.8491 - lr: 0.0010
Epoch 6/2000
383/383 [==============================] - 4s 10ms/step - loss: 0.4406 - accuracy: 0.8667 - val_loss: 0.4313 - val_accuracy: 0.8682 - lr: 0.0010
Epoch 7/2000
383/383 [==============================

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn2.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn2.h5')
model_eval()

0.895

In [ ]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################

model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

model.fit(
          train_data_2,
          train_labels_2,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/2000
383/383 [==============================] - 5s 11ms/step - loss: 1.3102 - accuracy: 0.5779 - val_loss: 0.7070 - val_accuracy: 0.7507 - lr: 0.0010
Epoch 2/2000
383/383 [==============================] - 4s 10ms/step - loss: 0.6338 - accuracy: 0.7814 - val_loss: 0.5697 - val_accuracy: 0.8127 - lr: 0.0010
Epoch 3/2000
383/383 [==============================] - 4s 10ms/step - loss: 0.5559 - accuracy: 0.8154 - val_loss: 0.5158 - val_accuracy: 0.8337 - lr: 0.0010
Epoch 4/2000
383/383 [==============================] - 4s 10ms/step - loss: 0.5010 - accuracy: 0.8418 - val_loss: 0.4802 - val_accuracy: 0.8527 - lr: 0.0010
Epoch 5/2000
383/383 [==============================] - 4s 10ms/step - loss: 0.4655 - accuracy: 0.8553 - val_loss: 0.4506 - val_accuracy: 0.8637 - lr: 0.0010
Epoch 6/2000
383/383 [==============================] - 4s 10ms/step - loss: 0.4408 - accuracy: 0.8658 - val_loss: 0.4524 - val_accuracy: 0.8623 - lr: 0.0010
Epoch 7/2000
383/383 [==============================

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no.h5')
model_eval()

0.8908

#### Resnet

In [31]:
from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

def residual_block(x: Tensor, downsample: bool, filters: int, kernel_size: int = 3) -> Tensor:
    y = Conv2D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

def create_res_net():
    
    inputs = Input(shape=(28, 28, 1))
    num_filters = 64
    
    t = BatchNormalization()(inputs)
    t = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)
    t = PermaDropout(0.5)(t)
    
    num_blocks_list = [2, 5, 5, 2]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            t = residual_block(t, downsample=(j==0 and i!=0), filters=num_filters)
        num_filters *= 2
        t = PermaDropout(0.5)(t)
    
    t = AveragePooling2D(4)(t)
    t = Flatten()(t)
    outputs = Dense(10, activation='softmax')(t)
    
    model = Model(inputs, outputs)

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
model = create_res_net()
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')

model.fit(
    x=train_data_1,
    y=train_labels_1,
    epochs=500,
    verbose=1,
    validation_split=0.3,
    batch_size=64,
    callbacks=[lr_reducer, early_stopper]
)

Epoch 1/500
383/383 [==============================] - 125s 310ms/step - loss: 0.6916 - accuracy: 0.7503 - val_loss: 1.3493 - val_accuracy: 0.6064 - lr: 0.0010
Epoch 2/500
383/383 [==============================] - 116s 303ms/step - loss: 0.3944 - accuracy: 0.8613 - val_loss: 0.5656 - val_accuracy: 0.8303 - lr: 0.0010
Epoch 3/500
383/383 [==============================] - 116s 303ms/step - loss: 0.3401 - accuracy: 0.8839 - val_loss: 0.3568 - val_accuracy: 0.8836 - lr: 0.0010
Epoch 4/500
383/383 [==============================] - 116s 303ms/step - loss: 0.3054 - accuracy: 0.8961 - val_loss: 0.4000 - val_accuracy: 0.8769 - lr: 0.0010
Epoch 5/500
383/383 [==============================] - 116s 303ms/step - loss: 0.2870 - accuracy: 0.9028 - val_loss: 0.4595 - val_accuracy: 0.8676 - lr: 0.0010
Epoch 6/500
383/383 [==============================] - 116s 303ms/step - loss: 0.2667 - accuracy: 0.9107 - val_loss: 0.3334 - val_accuracy: 0.8914 - lr: 0.0010
Epoch 7/500
383/383 [===================

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_resnet_a1.h5')

In [ ]:
model_eval()

In [ ]:
model = create_res_net()
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')

model.fit(
    x=train_data_2,
    y=train_labels_2,
    epochs=500,
    verbose=1,
    validation_split=0.3,
    batch_size=64,
    callbacks=[lr_reducer, early_stopper]
)

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_resnet_b1.h5')

In [ ]:
model_eval()

# 남은 데이터 라벨링하기


In [ ]:
model1 = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_resnet_a1.h5')
model2 = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_resnet_b1.h5')

In [ ]:
unlab_label_1 = model1.predict(unlab_data_1)

In [ ]:
unlab_label_2 = model2.predict(unlab_data_2)

In [ ]:
train_data_1 = np.concatenate([train_data_1, unlab_data_1], axis=0)
train_labels_1 = np.concatenate([train_labels_1, unlab_label_1], axis=0)

In [ ]:
train_data_2 = np.concatenate([train_data_2, unlab_data_2], axis=0)
train_labels_2 = np.concatenate([train_labels_2, unlab_label_2], axis=0)

In [ ]:
train_data_1.shape

In [ ]:
train_data_2.shape

#### cnn

In [ ]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)


model.fit(
          train_data_1,
          train_labels_1,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/2000
657/657 [==============================] - 8s 11ms/step - loss: 1.0349 - accuracy: 0.6429 - val_loss: 0.6362 - val_accuracy: 0.7989 - lr: 0.0010
Epoch 2/2000
657/657 [==============================] - 7s 10ms/step - loss: 0.5660 - accuracy: 0.8134 - val_loss: 0.5578 - val_accuracy: 0.8264 - lr: 0.0010
Epoch 3/2000
657/657 [==============================] - 7s 11ms/step - loss: 0.5058 - accuracy: 0.8400 - val_loss: 0.4932 - val_accuracy: 0.8588 - lr: 0.0010
Epoch 4/2000
657/657 [==============================] - 7s 11ms/step - loss: 0.4663 - accuracy: 0.8589 - val_loss: 0.4715 - val_accuracy: 0.8762 - lr: 0.0010
Epoch 5/2000
657/657 [==============================] - 7s 11ms/step - loss: 0.4360 - accuracy: 0.8760 - val_loss: 0.4621 - val_accuracy: 0.8806 - lr: 0.0010
Epoch 6/2000
657/657 [==============================] - 7s 10ms/step - loss: 0.4188 - accuracy: 0.8819 - val_loss: 0.4362 - val_accuracy: 0.8884 - lr: 0.0010
Epoch 7/2000
657/657 [==============================

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn3.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn3.h5')
model_eval()

0.8952

In [ ]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)


model.fit(
          train_data_2,
          train_labels_2,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/2000
657/657 [==============================] - 8s 11ms/step - loss: 1.0688 - accuracy: 0.6314 - val_loss: 0.6529 - val_accuracy: 0.7936 - lr: 0.0010
Epoch 2/2000
657/657 [==============================] - 7s 11ms/step - loss: 0.5860 - accuracy: 0.8049 - val_loss: 0.5950 - val_accuracy: 0.8197 - lr: 0.0010
Epoch 3/2000
657/657 [==============================] - 7s 10ms/step - loss: 0.5214 - accuracy: 0.8291 - val_loss: 0.5220 - val_accuracy: 0.8512 - lr: 0.0010
Epoch 4/2000
657/657 [==============================] - 7s 11ms/step - loss: 0.4783 - accuracy: 0.8530 - val_loss: 0.4864 - val_accuracy: 0.8728 - lr: 0.0010
Epoch 5/2000
657/657 [==============================] - 7s 11ms/step - loss: 0.4484 - accuracy: 0.8699 - val_loss: 0.4764 - val_accuracy: 0.8792 - lr: 0.0010
Epoch 6/2000
657/657 [==============================] - 7s 11ms/step - loss: 0.4260 - accuracy: 0.8772 - val_loss: 0.4519 - val_accuracy: 0.8859 - lr: 0.0010
Epoch 7/2000
657/657 [==============================

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no2.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no2.h5')
model_eval()

0.8944

#### resnet

In [ ]:
from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

def residual_block(x: Tensor, downsample: bool, filters: int, kernel_size: int = 3) -> Tensor:
    y = Conv2D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

def create_res_net():
    
    inputs = Input(shape=(28, 28, 1))
    num_filters = 64
    
    t = BatchNormalization()(inputs)
    t = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)
    t = PermaDropout(0.5)(t)
    
    num_blocks_list = [2, 5, 5, 2]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            t = residual_block(t, downsample=(j==0 and i!=0), filters=num_filters)
        num_filters *= 2
        t = PermaDropout(0.5)(t)
    
    t = AveragePooling2D(4)(t)
    t = Flatten()(t)
    outputs = Dense(10, activation='softmax')(t)
    
    model = Model(inputs, outputs)

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
model = create_res_net()
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')

model.fit(
    x=train_data_1,
    y=train_labels_1,
    epochs=500,
    verbose=1,
    validation_split=0.3,
    batch_size=64,
    callbacks=[lr_reducer, early_stopper]
)

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_resnet_a2.h5')

In [ ]:
model_eval()

In [ ]:
model = create_res_net()
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')

model.fit(
    x=train_data_2,
    y=train_labels_2,
    epochs=500,
    verbose=1,
    validation_split=0.3,
    batch_size=64,
    callbacks=[lr_reducer, early_stopper]
)

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_resnet_b2.h5')

In [ ]:
model_eval()